# The Workspace

The workspace pattern provides a shared, persistent file system where agents externalize artifacts too large for the context window. Agents see sandbox paths (`/workspace/...`) while actual files are stored in isolated directories per user and session.

In [ ]:
from pathlib import PurePosixPath
import json

from agentic_patterns.core.workspace import (
    workspace_to_host_path,
    write_to_workspace,
    read_from_workspace,
    list_workspace_files,
    WorkspaceError,
)
from agentic_patterns.core.user_session import (
    set_user_session,
    get_user_id,
    get_session_id,
)
from agentic_patterns.core.config.config import WORKSPACE_DIR
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.utils import relative_to_home

## Identity Setup

The workspace resolves user/session identity from contextvars. Call `set_user_session()` once at the request boundary (middleware, MCP handler, etc.) and all downstream code picks it up automatically.

In [ ]:
set_user_session("alice", "session_001")

print(f"User: {get_user_id()}")
print(f"Session: {get_session_id()}")
print(f"Workspace base: {relative_to_home(WORKSPACE_DIR)}")

## Path Translation

Helper functions convert between sandbox paths (what agents see) and host paths (actual filesystem). User/session identity is resolved automatically from contextvars.

In [ ]:
sandbox_path = "/workspace/reports/analysis.json"
host_path = workspace_to_host_path(PurePosixPath(sandbox_path))

print(f"Agent sees:    {sandbox_path}")
print(f"Actual file:   {relative_to_home(host_path)}")

## Tool Pattern: Write Large Output, Return Summary

When a tool produces large output, it writes the full result to the workspace and returns a concise summary with the file path. The agent's context stays small.

In [ ]:
def analyze_dataset(query: str) -> str:
    """Analyze data and save results to workspace."""
    result = {
        "query": query,
        "row_count": 50000,
        "statistics": {"mean": 42.5, "std": 12.3, "min": 0.1, "max": 99.8},
        "data": [{"id": i, "value": i * 0.1} for i in range(1000)],
    }

    output_path = "/workspace/analysis/result.json"
    write_to_workspace(output_path, json.dumps(result, indent=2))

    return f"""Analysis complete. Rows: {result["row_count"]}, Mean: {result["statistics"]["mean"]}
Full results: {output_path}"""

In [ ]:
summary = analyze_dataset("SELECT * FROM metrics")
print(summary)

# Verify file exists on disk
host_path = workspace_to_host_path(PurePosixPath("/workspace/analysis/result.json"))
print(f"\nFile size: {host_path.stat().st_size} bytes")

## Agent with Workspace Tools

Tools no longer need closures to capture context. Since user/session is in contextvars, tools just call workspace functions directly.

In [ ]:
def search_data(query: str) -> str:
    """Search dataset and save results to workspace."""
    matches = [
        {"id": i, "name": f"item_{i}", "score": 0.9 - i * 0.01} for i in range(500)
    ]

    output_path = "/workspace/search_results.json"
    write_to_workspace(output_path, json.dumps(matches))

    return f"Found {len(matches)} matches. Top 3: {matches[:3]}. Full results: {output_path}"


def read_file(path: str) -> str:
    """Read a file from the workspace."""
    return read_from_workspace(path)


tools = [search_data, read_file]

In [ ]:
agent = get_agent(tools=tools)

prompt = "Search for sensor data and tell me how many results were found."
agent_run, nodes = await run_agent(agent, prompt, verbose=True)

print(f"\nAnswer: {agent_run.result.output}")

In [ ]:
# Full data persists in workspace
print("Files in workspace:")
for f in list_workspace_files("*.json"):
    host_path = workspace_to_host_path(PurePosixPath(f))
    print(f"  {f} ({host_path.stat().st_size} bytes)")

## User Isolation

Each user/session gets an isolated directory. The identity extractor determines which workspace to use based on the request context.

In [ ]:
# Switch to Bob's context
set_user_session("bob", "session_001")
write_to_workspace("/workspace/secret.txt", "Bob's private data")
bob_path = workspace_to_host_path(PurePosixPath("/workspace/secret.txt"))

# Switch back to Alice
set_user_session("alice", "session_001")
alice_path = workspace_to_host_path(PurePosixPath("/workspace/secret.txt"))

print(f"Bob's file:   {relative_to_home(bob_path)}")
print(f"Alice's file: {relative_to_home(alice_path)}")

## Security: Path Traversal Prevention

In [ ]:
try:
    workspace_to_host_path(PurePosixPath("/workspace/../../../etc/passwd"))
except WorkspaceError as e:
    print(f"Blocked: {e}")

try:
    workspace_to_host_path(PurePosixPath("/etc/passwd"))
except WorkspaceError as e:
    print(f"Blocked: {e}")